In [1]:
from pyspark.sql import SparkSession
import getpass 
username=getpass.getuser()
spark=SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

#### 1. create a dataframe with proper datatypes 

In [2]:
customer_schema = 'member_id string, emp_title string, emp_length string, home_ownership string, annual_inc float, addr_state string, zip_code string, country string, grade string, sub_grade string, verification_status string, tot_hi_cred_lim float, application_type string, annual_inc_joint float, verification_status_joint string'

In [3]:
customers_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(customer_schema) \
.load("/public/trendytech/lendingclubproject/raw/customers_data_csv")

In [4]:
customers_raw_df

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
b59d80da191f5b573...,null,null,RENT,50000.0,OR,973xx,USA,A,A5,Source Verified,8600.0,Individual,null,null
202d9f56ecb7c3bc9...,police officer,7 years,OWN,85000.0,TX,799xx,USA,A,A5,Source Verified,272384.0,Individual,null,null
e5a140c0922b554b9...,community living ...,6 years,RENT,48000.0,NY,146xx,USA,B,B2,Source Verified,85092.0,Individual,null,null
e12aefc548f750777...,Office,10+ years,OWN,33000.0,CT,067xx,USA,F,F1,Verified,7100.0,Individual,null,null
1b3a50d854fbbf97e...,Special Tooling I...,10+ years,MORTGAGE,81000.0,TX,791xx,USA,E,E5,Verified,190274.0,Individual,null,null
1c4329e5f17697127...,Mine ops tech 6,2 years,MORTGAGE,68000.0,AZ,855xx,USA,C,C3,Not Verified,182453.0,Individual,null,null
5026c86ad983175eb...,caregiver,4 years,RENT,76020.0,WA,993xx,USA,C,C2,Source Verified,15308.0,Individual,null,null
9847d8c1e9d0b2084...,null,null,OWN,65000.0,IL,624xx,USA,E,E3,Verified,128800.0,Individual,null,null
8340dbe1adea41fb4...,Vice President Re...,8 years,MORTGAGE,111000.0,CT,063xx,USA,A,A1,Not Verified,343507.0,Individual,null,null
d4de0de3ab7d79ad4...,FOREMAN,10+ years,MORTGAGE,67000.0,WA,992xx,USA,G,G2,Verified,211501.0,Individual,null,null


In [5]:
customers_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- annual_inc_joint: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)



#### 2. Rename a few columns

In [6]:
customer_df_renamed = customers_raw_df.withColumnRenamed("annual_inc", "annual_income") \
.withColumnRenamed("addr_state", "address_state") \
.withColumnRenamed("zip_code", "address_zipcode") \
.withColumnRenamed("country", "address_country") \
.withColumnRenamed("tot_hi_credit_lim", "total_high_credit_limit") \
.withColumnRenamed("annual_inc_joint", "join_annual_income")

In [7]:
customer_df_renamed

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,join_annual_income,verification_status_joint
b59d80da191f5b573...,null,null,RENT,50000.0,OR,973xx,USA,A,A5,Source Verified,8600.0,Individual,null,null
202d9f56ecb7c3bc9...,police officer,7 years,OWN,85000.0,TX,799xx,USA,A,A5,Source Verified,272384.0,Individual,null,null
e5a140c0922b554b9...,community living ...,6 years,RENT,48000.0,NY,146xx,USA,B,B2,Source Verified,85092.0,Individual,null,null
e12aefc548f750777...,Office,10+ years,OWN,33000.0,CT,067xx,USA,F,F1,Verified,7100.0,Individual,null,null
1b3a50d854fbbf97e...,Special Tooling I...,10+ years,MORTGAGE,81000.0,TX,791xx,USA,E,E5,Verified,190274.0,Individual,null,null
1c4329e5f17697127...,Mine ops tech 6,2 years,MORTGAGE,68000.0,AZ,855xx,USA,C,C3,Not Verified,182453.0,Individual,null,null
5026c86ad983175eb...,caregiver,4 years,RENT,76020.0,WA,993xx,USA,C,C2,Source Verified,15308.0,Individual,null,null
9847d8c1e9d0b2084...,null,null,OWN,65000.0,IL,624xx,USA,E,E3,Verified,128800.0,Individual,null,null
8340dbe1adea41fb4...,Vice President Re...,8 years,MORTGAGE,111000.0,CT,063xx,USA,A,A1,Not Verified,343507.0,Individual,null,null
d4de0de3ab7d79ad4...,FOREMAN,10+ years,MORTGAGE,67000.0,WA,992xx,USA,G,G2,Verified,211501.0,Individual,null,null


In [8]:
from pyspark.sql.functions import current_timestamp

#### 3. insert a new column named as ingestion date(current time)

In [31]:
customers_df_ingestd = customer_df_renamed.withColumn("ingest_date", current_timestamp())

In [32]:
customers_df_ingestd

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,join_annual_income,verification_status_joint,ingest_date
b59d80da191f5b573...,null,null,RENT,50000.0,OR,973xx,USA,A,A5,Source Verified,8600.0,Individual,null,null,2023-12-31 00:46:...
202d9f56ecb7c3bc9...,police officer,7 years,OWN,85000.0,TX,799xx,USA,A,A5,Source Verified,272384.0,Individual,null,null,2023-12-31 00:46:...
e5a140c0922b554b9...,community living ...,6 years,RENT,48000.0,NY,146xx,USA,B,B2,Source Verified,85092.0,Individual,null,null,2023-12-31 00:46:...
e12aefc548f750777...,Office,10+ years,OWN,33000.0,CT,067xx,USA,F,F1,Verified,7100.0,Individual,null,null,2023-12-31 00:46:...
1b3a50d854fbbf97e...,Special Tooling I...,10+ years,MORTGAGE,81000.0,TX,791xx,USA,E,E5,Verified,190274.0,Individual,null,null,2023-12-31 00:46:...
1c4329e5f17697127...,Mine ops tech 6,2 years,MORTGAGE,68000.0,AZ,855xx,USA,C,C3,Not Verified,182453.0,Individual,null,null,2023-12-31 00:46:...
5026c86ad983175eb...,caregiver,4 years,RENT,76020.0,WA,993xx,USA,C,C2,Source Verified,15308.0,Individual,null,null,2023-12-31 00:46:...
9847d8c1e9d0b2084...,null,null,OWN,65000.0,IL,624xx,USA,E,E3,Verified,128800.0,Individual,null,null,2023-12-31 00:46:...
8340dbe1adea41fb4...,Vice President Re...,8 years,MORTGAGE,111000.0,CT,063xx,USA,A,A1,Not Verified,343507.0,Individual,null,null,2023-12-31 00:46:...
d4de0de3ab7d79ad4...,FOREMAN,10+ years,MORTGAGE,67000.0,WA,992xx,USA,G,G2,Verified,211501.0,Individual,null,null,2023-12-31 00:46:...


#### 4. Remove complete duplicate rows

In [33]:
customers_df_ingestd.count()

2260701

In [34]:
customers_distinct = customers_df_ingestd.distinct()

In [35]:
customers_distinct.count()

2260638

In [36]:
customers_distinct.createOrReplaceTempView("customers")

In [37]:
spark.sql("select * from customers")

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,join_annual_income,verification_status_joint,ingest_date
7c27017e8258b64e7...,null,null,RENT,14000.0,CA,907xx,USA,C,C5,Source Verified,6436.0,Individual,null,null,2023-12-31 00:46:...
d46df6e7bc3c62431...,Legal Administrat...,4 years,RENT,35360.0,CA,902xx,USA,G,G2,Verified,16963.0,Individual,null,null,2023-12-31 00:46:...
959d45e7dfcabaf78...,Access Control Sp...,7 years,RENT,52000.0,DC,200xx,USA,C,C3,Not Verified,83154.0,Individual,null,null,2023-12-31 00:46:...
96172517474d8f69c...,OWNER,5 years,RENT,57000.0,FL,328xx,USA,D,D1,Source Verified,45721.0,Individual,null,null,2023-12-31 00:46:...
bc73ad92a5de0ec20...,Business account ...,7 years,MORTGAGE,100000.0,FL,335xx,USA,B,B4,Source Verified,139418.0,Individual,null,null,2023-12-31 00:46:...
9185f33e48e2b1129...,Arch/ Eng Tech,10+ years,MORTGAGE,78000.0,IL,605xx,USA,C,C5,Not Verified,40200.0,Individual,null,null,2023-12-31 00:46:...
2f7df5b7d6d3b1050...,Supervisor,7 years,OWN,65000.0,PA,194xx,USA,B,B4,Source Verified,264200.0,Individual,null,null,2023-12-31 00:46:...
254b724ea7e902228...,Driver,10+ years,RENT,45000.0,CA,906xx,USA,C,C1,Source Verified,28301.0,Individual,null,null,2023-12-31 00:46:...
40daad042214e292d...,IT Training Manager,2 years,RENT,151000.0,NY,100xx,USA,D,D1,Verified,107515.0,Individual,null,null,2023-12-31 00:46:...
403f35d892e515317...,Mortgage,10+ years,RENT,39520.0,SC,292xx,USA,C,C5,Source Verified,64463.0,Individual,null,null,2023-12-31 00:46:...


#### 5. Remove the rows where annual_income is null

In [38]:
spark.sql("select count(*) from customers where annual_income is null")

count(1)
5


In [39]:
customers_income_filtered = spark.sql("select * from customers where annual_income is not null")

In [40]:
customers_income_filtered.createOrReplaceTempView("customers")

### 6. convert emp_length to integer

In [42]:
spark.sql("select distinct(emp_length) from customers")

emp_length
9 years
5 years
null
1 year
2 years
7 years
8 years
4 years
6 years
3 years


In [43]:
from pyspark.sql.functions import regexp_replace, col

In [44]:
customers_emplength_cleaned = customers_income_filtered.withColumn("emp_length", regexp_replace(col("emp_length"), "(\D)",""))

In [45]:
customers_emplength_cleaned

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,join_annual_income,verification_status_joint,ingest_date
52817b840418f1c32...,Bus Operator/Mech...,10,MORTGAGE,49000.0,WV,258xx,USA,A,A4,Source Verified,208200.0,Individual,null,null,2023-12-31 00:47:...
9e2995bead58b0eb7...,Regional BIM Manager,2,RENT,105000.0,CA,900xx,USA,B,B5,Source Verified,65798.0,Individual,null,null,2023-12-31 00:47:...
ba04c4d49e7f1ac8d...,Train Operator,6,MORTGAGE,98000.0,NY,110xx,USA,B,B1,Not Verified,193981.0,Individual,null,null,2023-12-31 00:47:...
bbbfcfae3e7bc1b77...,IT Consultant,2,RENT,65000.0,CA,913xx,USA,B,B4,Not Verified,63829.0,Individual,null,null,2023-12-31 00:47:...
023653d46456279f0...,Care giver,1,RENT,45000.0,IL,622xx,USA,C,C4,Verified,48000.0,Individual,null,null,2023-12-31 00:47:...
b9b636c972b9067bd...,QA Engineer,5,MORTGAGE,86000.0,TX,750xx,USA,B,B5,Not Verified,359396.0,Individual,null,null,2023-12-31 00:47:...
a4d6a1fd86af72435...,night manger,5,OWN,43000.0,TN,370xx,USA,A,A3,Not Verified,94260.0,Individual,null,null,2023-12-31 00:47:...
87404ee54b8fed86a...,manager,2,RENT,45600.0,NV,891xx,USA,D,D1,Source Verified,52817.0,Individual,null,null,2023-12-31 00:47:...
ca0bad11080d19e35...,AVP/EDD Manager,3,RENT,78000.0,UT,840xx,USA,C,C5,Source Verified,75200.0,Individual,null,null,2023-12-31 00:47:...
4125d1f111d2aff4f...,Assistant Manager,8,RENT,69000.0,MA,021xx,USA,A,A5,Source Verified,18500.0,Individual,null,null,2023-12-31 00:47:...


In [46]:
from pyspark.sql import SparkSession
import getpass 
username=getpass.getuser()
spark=SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [47]:
customers_emplength_cleaned

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,join_annual_income,verification_status_joint,ingest_date
07919a66fb19526a5...,Infantryman,8,RENT,25000.0,FL,328xx,USA,C,C3,Not Verified,7000.0,Individual,null,null,2023-12-31 00:47:...
774d7c5c678dc90a0...,Borrower Services...,1,RENT,27000.0,WI,544xx,USA,D,D1,Not Verified,121804.0,Individual,null,null,2023-12-31 00:47:...
3049004a52aa8051f...,Executive,10,MORTGAGE,85000.0,IL,625xx,USA,B,B5,Source Verified,197340.0,Individual,null,null,2023-12-31 00:47:...
446d4fd8e01e48f62...,"Owner, Broker",1,RENT,65385.0,NC,276xx,USA,D,D4,Source Verified,50249.0,Individual,null,null,2023-12-31 00:47:...
1e780774010b509f6...,Installer,6,RENT,30000.0,OR,972xx,USA,B,B4,Source Verified,12850.0,Individual,null,null,2023-12-31 00:47:...
037b490c131c26b6a...,Tech,2,MORTGAGE,60000.0,NY,133xx,USA,F,F5,Not Verified,106937.0,Individual,null,null,2023-12-31 00:47:...
20bc0d74f204028af...,Teacher,2,OWN,41500.0,SD,577xx,USA,E,E1,Not Verified,104216.0,Individual,null,null,2023-12-31 00:47:...
40a68bebc76469d0f...,"NCOIC,A2 Executiv...",10,RENT,69000.0,VA,221xx,USA,F,F2,Not Verified,255455.0,Individual,null,null,2023-12-31 00:47:...
3786f670168c9fc25...,Training Store Ma...,3,MORTGAGE,65632.06,WA,986xx,USA,C,C1,Source Verified,70355.0,Individual,null,null,2023-12-31 00:47:...
e57b56b9dd317a52a...,Program Assistant,10,MORTGAGE,60000.0,CA,930xx,USA,D,D2,Source Verified,67045.0,Individual,null,null,2023-12-31 00:47:...


In [48]:
customers_emplength_cleaned.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- address_state: string (nullable = true)
 |-- address_zipcode: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- join_annual_income: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [50]:
customers_emplength_casted = customers_emplength_cleaned.withColumn("emp_length", customers_emplength_cleaned.emp_length.cast('integer'))

In [51]:
customers_emplength_casted

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,join_annual_income,verification_status_joint,ingest_date
81096406c34046abb...,CSR Supervisor,10,MORTGAGE,25600.0,OH,457xx,USA,D,D2,Source Verified,76451.0,Individual,null,null,2023-12-31 00:47:...
e72695f14a5e29a1b...,Food and Beverage...,2,MORTGAGE,50000.0,TX,765xx,USA,A,A2,Not Verified,148775.0,Individual,null,null,2023-12-31 00:47:...
ebec3cc63af0b4fab...,Rehabilitation Th...,10,RENT,77556.0,CA,932xx,USA,C,C1,Source Verified,84315.0,Individual,null,null,2023-12-31 00:47:...
14cb1f4e485b9bb62...,Technical Editor,1,RENT,95000.0,VA,220xx,USA,A,A5,Source Verified,65895.0,Individual,null,null,2023-12-31 00:47:...
fe6eaf055aed9a37f...,Business Analyst,5,MORTGAGE,102000.0,GA,317xx,USA,B,B3,Verified,423223.0,Individual,null,null,2023-12-31 00:47:...
400031c5ce066fa59...,Medical Device In...,4,RENT,75000.0,TX,770xx,USA,A,A3,Not Verified,250081.0,Individual,null,null,2023-12-31 00:47:...
6ae4e8126464f127c...,Agent,10,MORTGAGE,90000.0,FL,349xx,USA,A,A2,Source Verified,215494.0,Individual,null,null,2023-12-31 00:47:...
7bdf28e0a83588faf...,Sr. Project Manager,7,RENT,200000.0,CA,926xx,USA,D,D4,Source Verified,243634.0,Individual,null,null,2023-12-31 00:47:...
4de7aec3c24960c15...,sales manager,10,MORTGAGE,62000.0,TX,797xx,USA,B,B5,Not Verified,208982.0,Individual,null,null,2023-12-31 00:47:...
0c3938378a6db8da4...,Associate Client ...,1,RENT,50000.0,WA,980xx,USA,B,B1,Not Verified,51535.0,Individual,null,null,2023-12-31 00:47:...


In [52]:
customers_emplength_casted.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: integer (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- address_state: string (nullable = true)
 |-- address_zipcode: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- join_annual_income: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



#### 7. we need to replace all the nulls in emp_length column with average of this column

In [53]:
customers_emplength_casted.filter("emp_length is null").count()

146903

In [54]:
customers_emplength_casted.createOrReplaceTempView("customers")

In [ ]:
avg_emp_length = spark.sql("select floor(avg(emp_length)) as avg_emp_length from customers").collect()

In [ ]:
print(avg_emp_length)

In [32]:
avg_emp_duration = avg_emp_length[0][0]

In [33]:
print(avg_emp_duration)

6


In [34]:
customers_emplength_replaced = customers_emplength_casted.na.fill(avg_emp_duration, subset=['emp_length'])

In [35]:
customers_emplength_replaced

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,join_annual_income,verification_status_joint,ingest_date
e5324e6ec6668c91a...,forman,10,RENT,100000.0,NJ,074xx,USA,C,C2,Verified,50640.0,Individual,null,null,2023-10-04 05:38:...
a9a8fcbcde5d446bb...,Lead clerk,10,MORTGAGE,55000.0,CA,958xx,USA,D,D3,Not Verified,266596.0,Individual,null,null,2023-10-04 05:38:...
b711748d9cacaeb06...,SECONDARY MARKETI...,1,RENT,35360.0,CA,928xx,USA,B,B5,Source Verified,16000.0,Individual,null,null,2023-10-04 05:38:...
00b2d0e6089cb33ed...,Police Sergeant,9,RENT,62000.0,VA,236xx,USA,C,C4,Not Verified,174524.0,Individual,null,null,2023-10-04 05:38:...
7f0f9bb5da43380c6...,laborer,1,RENT,38400.0,IN,461xx,USA,D,D3,Source Verified,7300.0,Individual,null,null,2023-10-04 05:38:...
13663735f3a3e3c03...,Health Unit Coord...,10,RENT,50000.0,OH,441xx,USA,B,B1,Not Verified,67458.0,Individual,null,null,2023-10-04 05:38:...
f7d087e0bd0e26a95...,Compliance Manager,2,MORTGAGE,61200.0,OK,731xx,USA,C,C2,Verified,277246.0,Individual,null,null,2023-10-04 05:38:...
bbaf77dea69ad59ba...,DIRECT CARE WORKER,3,OWN,25000.0,MO,654xx,USA,C,C3,Source Verified,29198.0,Individual,null,null,2023-10-04 05:38:...
3d5e1eb7b1a075d7c...,Office Manager,6,RENT,37900.0,HI,967xx,USA,D,D2,Verified,72700.0,Individual,null,null,2023-10-04 05:38:...
612623c0c3a9d059b...,null,6,MORTGAGE,65000.0,WV,249xx,USA,C,C3,Source Verified,42392.0,Individual,null,null,2023-10-04 05:38:...


In [36]:
customers_emplength_replaced.filter("emp_length is null").count()

0

#### 8. Clean the address_state(it should be 2 characters only),replace all others with NA

In [37]:
customers_emplength_replaced.createOrReplaceTempView("customers")

In [38]:
spark.sql("select distinct(address_state) from customers")

address_state
Helping Kenya's D...
223xx
175 (total projec...
AZ
SC
I am 56 yrs. old ...
"so Plan """"C"""" is ..."
financially I mad...
but no one will l...
LA


In [39]:
spark.sql("select count(address_state) from customers where length(address_state)>2")

count(address_state)
254


In [40]:
from pyspark.sql.functions import when, col, length

In [41]:
customers_state_cleaned = customers_emplength_replaced.withColumn(
    "address_state",
    when(length(col("address_state"))> 2, "NA").otherwise(col("address_state"))
)

In [42]:
customers_state_cleaned

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,join_annual_income,verification_status_joint,ingest_date
d16e2f99a726e8326...,Sr. Network Desig...,8,MORTGAGE,82000.0,GA,301xx,USA,B,B4,Source Verified,427673.0,Individual,null,null,2023-10-04 05:38:...
39c7ce3e4838895b4...,VP Marketing,10,RENT,155000.0,NY,105xx,USA,B,B3,Not Verified,92200.0,Individual,null,null,2023-10-04 05:38:...
b061c1f3197c674d1...,Driver,9,RENT,93000.0,HI,967xx,USA,C,C5,Verified,130019.0,Individual,null,null,2023-10-04 05:38:...
c728d0bd111b0240e...,Transport Driver ...,10,MORTGAGE,130000.0,TX,775xx,USA,C,C5,Source Verified,172012.0,Individual,null,null,2023-10-04 05:38:...
4fe2c555c12ec7e02...,Office Manager,10,MORTGAGE,62000.0,WA,985xx,USA,F,F1,Verified,333602.0,Individual,null,null,2023-10-04 05:38:...
1e4d2744d484898a7...,Senior Accountant,10,OWN,74000.0,KS,662xx,USA,B,B3,Verified,324392.0,Individual,null,null,2023-10-04 05:38:...
4888e607570b21f6c...,5th Grade Teacher,10,MORTGAGE,72000.0,GA,398xx,USA,A,A2,Verified,213654.0,Individual,null,null,2023-10-04 05:38:...
03dc367806c43d74d...,purchaser,10,RENT,59000.0,CA,921xx,USA,B,B3,Verified,75835.0,Individual,null,null,2023-10-04 05:38:...
59af1f6dae2947b4a...,Engineer,10,MORTGAGE,110000.0,OH,453xx,USA,C,C4,Source Verified,296094.0,Individual,null,null,2023-10-04 05:38:...
4ec343b2e7c664944...,teachers aide,10,MORTGAGE,35000.0,MO,641xx,USA,F,F3,Verified,65428.0,Individual,null,null,2023-10-04 05:38:...


In [43]:
customers_state_cleaned.select("address_state").distinct()

address_state
AZ
SC
LA
MN
NJ
DC
OR
NA
VA
null


In [44]:
customers_state_cleaned.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv006277/lendingclubproject/raw/cleaned/customers_parquet") \
.save()

In [45]:
customers_state_cleaned.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv006277/lendingclubproject/raw/cleaned/customers_csv") \
.save()